In [19]:
#Here all imports will be done
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

In [9]:
#Importing Dataset
pos_tweets=twitter_samples.strings('positive_tweets.json')
neg_tweets=twitter_samples.strings('negative_tweets.json')
print("Some Positive tweets :")
for i in range(5):
    print(i+1,"- ",pos_tweets[i])
print()
print("Some Negative tweets :")
for i in range(5):
    print(i+1,"- ",neg_tweets[i])
print()
print("Total Positive tweets =",len(pos_tweets))
print("Total Negative tweets =",len(neg_tweets))

Some Positive tweets :
1 -  #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
2 -  @Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!
3 -  @DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!
4 -  @97sides CONGRATS :)
5 -  yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days

Some Negative tweets :
1 -  hopeless for tmr :(
2 -  Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(
3 -  @Hegelbon That heart sliding into the waste basket. :(
4 -  “@ketchBurning: I hate Japanese call him "bani" :( :(”

Me too
5 -  Dang starting next week I have "work" :(

Total Positive tweets = 5000
Total Negative tweets = 5000


In [36]:
#Pre-Processing
def pre_process(tweets):
    # (http[s]{0,1}://(\S)*)|(www\.[\S]+\.[\w]{1,6}) -> for matching url
    r1=re.compile(r'(http[s]{0,1}://(\S)*)|(www\.[\S]+\.[\w]{1,6})')
    # (^rt(\s)+)|([\s]rt[\s]*) -> for matching retweet(rt) at start or in between ...
    r2=re.compile(r'(^rt(\s)+)|([\s]rt[\s]*)')
    # @(\S)+ -> for matching @username ...
    r3=re.compile(r'@(\S)+')
    # #(\S)+ ->for matching #hashtags
    r4=re.compile(r'#(\S)+')
    # [^\w\s]+ -> for matching non word characters like symbols,emoticons,punctuations
    r5=re.compile(r'[^\w\s]+')
    k=1
    stopword=stopwords.words('english') #imported a list of stopwords
    #print(stopword)
    processed_tweets=[]
    for tweet in tweets:
        tweet=tweet.lower() #converting tweet to lower case
        tweet=re.sub(r1," ",tweet)
        tweet=re.sub(r2," ",tweet,re.M)
        tweet=re.sub(r3," ",tweet)
        tweet=re.sub(r4," ",tweet)
        tweet=re.sub(r5," ",tweet)
        tweet=' '.join(tweet.split())
        #k+=1
        tokens=word_tokenize(tweet) #tokenization
        #removing stopwords
        filtered_words=[]
        for token in tokens:
            if token not in stopword:
                filtered_words.append(token)
        #stemming
        stemmed_words=[]
        ps=PorterStemmer()
        for word in filtered_words:
            stemmed_words.append(ps.stem(word))
        processed_tweet=' '.join(stemmed_words)
        processed_tweets.append(processed_tweet)
    return processed_tweets

In [38]:
pre_process(pos_tweets)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

['top engag member commun week',
 'hey jame odd pleas call contact centr 02392441234 abl assist mani thank',
 'listen last night bleed amaz track scotland',
 'congrat',
 'yeaaaah yippppi accnt verifi rqst succeed got blue tick mark fb profil 15 day',
 'one irresist',
 'like keep love custom wait long hope enjoy happi friday lwwf',
 'second thought enough time dd new short enter system sheep must buy',
 'jgh go bayan bye',
 'act mischiev call etl layer hous wareh app katamari well name impli p',
 'top influenc commun week',
 'love big juici selfi',
 'follow amp follow u back',
 'perfect alreadi know wait',
 'great new opportun junior triathlet age 12 13 gatorad seri get entri',
 'lay greet card rang print today love job',
 'friend lunch yummmm',
 'id conflict thank help screenshot work',
 'hi liv',
 'hello need know someth u fm twitter sure thing dm x',
 'top new follow commun week',
 'heard four season pretti dope penthous obv fun',
 'yeah suppos lol chat bit x',
 'hello get youth job 

In [52]:
for i in range(len(pos_tweets)):
    print(i+1,"- ",pos_tweets[i])

1 -  #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
2 -  @Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!
3 -  @DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!
4 -  @97sides CONGRATS :)
5 -  yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days
6 -  @BhaktisBanter @PallaviRuhail This one is irresistible :)
#FlipkartFashionFriday http://t.co/EbZ0L2VENM
7 -  We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI
8 -  @Impatientraider On second thought, there’s just not enough time for a DD :) But new shorts entering system. Sheep must be buying.
9 -  Jgh , but we have to go to Bayan :D bye
10 -  As an act of mischievousness, am calling the ETL layer of our in-house 

756 -  @calllmejay_ sounds good to me :) I'll see you Saturday ✨
757 -  Live night with two fams :) #bloodline #huskies
758 -  @CorderMrs you're welcome :)
759 -  The picture I'd like to see in my obituary advert :) #goofingaround #madness #mad #bollywood… https://t.co/7tyUragh2H
760 -  @WhirlpoolBathUK lovely giveaway! :) #FreebieFriday
761 -  @roofbsuzy @roofyuna @roofsinb @roofchaelin @roofaim @roofjackson  dah move on :))
762 -  There's nothing as cool as being totally over someone, No bitterness, anger or hatred towards them. Just pure indifference :)
763 -  when u wake up at 4 AM and the suite life of Zach and Cody is on :))))!!!!!!
764 -  @rich_bartram29 Oh, didn't know :) But yes, you can have an order delivered to your work address, once it's added to your address book ^AC
765 -  Achieving excellence in Music as a Music Producer is a Mind Boggling &amp; Fatiguing Job... Baareeq... :) :) :)
766 -  @bexlittle @AlisonWright8 @lindyrendy @Cloud2ltd @DotNetSolutions love it thanks 

1182 -  @MrsPandP Thank you :)
1183 -  I know I should be packing. But procrastinating is so much more fun. :) Plus I need some outfit sorting... http://t.co/ep39uOZsoM
1184 -  @ron_boll Hi! Saw who u follow and thought u might like "Dark" https://t.co/ZGFwvC5IxT .Plz let us know what u think :)
1185 -  Telling people at KFC to go buy "she's kinda hot" on iTunes :))) #dedicatedfan
@5SOS 💜💜💜💜
1186 -  @RealCrimeMag Don't be daft.  Teething troubles.  You're allowed :-)
1187 -  @thestylebox Huxley looks so cool with his basket. Good choice of favourite shop too. :) Ben.
1188 -  @RaruShopping Thank you, I've sent them an email :)
1189 -  Gamer Follow Train!
Follow Me
Retweet this
Follow all who retweeted
Gain active followers :) 120
1190 -  @S0N0S_ Happy Birthday :)
1191 -  Given Kepler 452b distance I propose  'Far Away' as a suitable name. Perhaps @PeterFHamilton1 fans can comment or suggest alternatives? :-)
1192 -  @AnneHole @YouTube love using this :)
1193 -  @gitawnchstr we made some

1794 -  Wanted sleep at that time,guess i got it...Morning :) @LADE_KRANE
1795 -  "@divarh15: @GraceGithakwa Seems like you go out alot" something like that..:)
1796 -  agayhippiehippy Follback ya :)
1797 -  @carondeslut I loved the song! Keep up the good work :)
1798 -  @onlynewfaces hi check our final design :) https://t.co/EXt098Yq1b
1799 -  I don’t care that I have to force myself to exercise even when my body is screaming with pain :)
1800 -  @FlowersandCards @katerina7711 hard to better a good ginger cat :))
1801 -  Hi BAM ! @BarsAndMelody 
Can you follow my bestfriend @969Horan696 ? 
She loves you a lot :) 
See you in Warsaw &lt;3 
Love you &lt;3 x28
1802 -  @marypcbuk I don't see things as theres only one way to teach someone and raise awareness. anyway .. time for some real work ::) @aral
1803 -  @IQAEuroGames GO GO FRANCE GO ! :)
1804 -  @americascup Do you have stage times for Portsmouth live? :) x
1805 -  @LivingLifeNottm @MummyandMonkeys thanks for linking :) x
1806 -  @Ar

2319 -  @sophieb666 I couldn't agree more :-)
2320 -  Soooo pretty :D https://t.co/QKGhVfeL6o
2321 -  @gothsirius @vltraviolet my main man jesus just looking out for me :)
2322 -  @halfmoonrun Got my ticket for Amsterdam! Can't wait to see you guys perform for the 3rd time :)
2323 -  @SuperEezy lets do it. You drink again? If not I'll both your drinks and mine :)
2324 -  Nextweek :) @zfzar
2325 -  sends booty pics 2 my bestfriend bcuz what is a bae :-)
2326 -  @Saeek Step is good option in that case ... I use that a lot . Stable and sturdy :) 
@Ktens @vindee
2327 -  @KingRappaDZappa Hi! Would you like an @imPastel concert? Let me know your city and country and I'll start working on it! Thanks :)
2328 -  @ollyofficial how are you Olly? :)
2329 -  lukkkee my birthday is in 3 days it would be an awesome birthday present to get your follow !!:) @Luke5SOS
2330 -  @priyashaki93 hehe..ok da..am leaving now..see u again.ensoi lunch and day..keep smiles n tc madam :))
2331 -  @montague_web  tha

2937 -  Big thanks to everyone! Couldn't have done it without all the support! :) @HIT_LondonCity @BaxterStorey @GBCookBook @Bschefacademy
2938 -  @keevacrampton @AineAbbott1998 I always forget to tag ppl:):):)
2939 -  @KageYashsa Probably yes :p
2940 -  @leedslwt this makes me so :)))
2941 -  @Oloni great :)
2942 -  How about you just have a nice convo like a normal person instead of posting how u feel about 200 of your closest twitter friends :-)
2943 -  @nidhik6907 hope U'll njoy followingg me :)
2944 -  I'm more active on @waTAOwat and I'm near to making this account private so... :D
2945 -  Close your eyes :))
2946 -  @itradeland that's one of mine. Button pusher. Out soon :)
2947 -  @SpringwellLife thank you so much for having us! We're glad you enjoyed it :-)
2948 -  @rebelwithany Thank you Pri. :)
2949 -  @DanGodOfWar thank you! :)
2950 -  @2turntsav I'm not gonna freak out :::::)
2951 -  I second that, simply stunning! :D https://t.co/E9q4mgeTz4
2952 -  OMG!! I can't wait for 

3528 -  @_pearlyyy well, kind of :) I still read it if the author is my favorite,otherwise,I tend to not read them, that's why I crave for more fics
3529 -  @dvntownsend I thought that you wanted to take  it slower for while.. :D very glad though
3530 -  @Arean91 @robinashe The Asian SJW, trying not to cry...:D
3531 -  #cakehamper baskets have arrived :) Best gonna be busy 2 get the finished products ready for you all to see #excited http://t.co/1jbVpcfnXB
3532 -  @flowerxmouth awh good :) why is he hated though? Haha.
3533 -  @WoahMitchel thank you :)
3534 -  @CocoaBrownTan @FuschiaEffect I'm glowing this morning! Yayyyy :))) happy Friday! Xx
3535 -  @ErnestMnisi @stuhlinho  Hello :), the original new mercedes hood badge http://t.co/EZp6TqdWor
3536 -  @mediatemple I'd love you to add my @mediatemple hosted site to your blog :) Please. All about my drone videos etc http://t.co/R6B1MIutgJ
3537 -  Next week we're blowing up chocolate :D
3538 -  Aww, yay I'm in the "Not Hated Group" and I

4195 -  @SasaRichardson @Stefbystef_ @Frgt10_Anthem sombrero high?? OKAY I'M LOST NOW :)) http://t.co/2VDPgb5EpK
4196 -  @osmosisness helloooo! i'm rose, nice to meet you :)
4197 -  @balboa_lesly Thank you for the screenshots po. Let me take care of this spammer for you. :) Before we proceed, c... https://t.co/x4ObmZFYUB
4198 -  @Manaal_M16 @yusra and perfect thing to pick :) #WSaleLove
4199 -  @AlliDavies1 @INKProducer @sugaredeggs thank you! @thedanperry did such an amazing job :)
4200 -  @storpey don't die, you're actually pretty entertaining :p
4201 -  @MLGJordanD congrats on 100k Man. Can't wait till I can hit that milestone too :)
4202 -  Morning all...Judith here to answer any District Council related questions today :-)
4203 -  @Itsjustizue see you tomorrow then :))))
4204 -  saw a comment on how midares gender is unknown and like
:) yeah you just keep saying that
4205 -  @mariaburtea @5SOS_Nightly wow you really hate me :))))
4206 -  @Floreani_Mgmt @DanOlsenMusic @UKOpenMic @B

4787 -  @Btigzy @McJamKeyper JoeRine scene :)))))
JOSHANE POWER TANDEM
4788 -  @MCunleashed :D I can't sleep until I need to. If I try I just lay in bed bored
4789 -  Stats for the day have arrived. 6 new followers and NO unfollowers :) via http://t.co/zWIln6s729.
4790 -  "I shouldn't b called a friend if I am not there when needed" :) ...https://t.co/lfSZl8UbXt
4791 -  Hello Ligao City, Albay! :-)

#Concert
#BCYC
#LNHS https://t.co/3Lzz3xvAYp
4792 -  goodnight ! i love Luke with all my heart :-) all my love 💜
4793 -  @KylieDeVille Happy Birthday!! Enjoy your vacation!:)
4794 -  @Satbains1 Congratulations Sat on your 3rd honorary degree from everyone at Alacer :)
4795 -  Hey Raspberry Island  : skelo_ghost :) Want to get iPh0ne 6 for FREE? Kindly check my bi0. Thx https://t.co/TO0kTgrY0y
4796 -  @aliaa08 @colors_infinity @karanjohar Thank u!!  :)
4797 -  @_Halie13_ thanks for being so great!:) ❤️😘
4798 -  @QueenBiankee You deserve all of that and madadagdagan pa yan :-) Congrats again 

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/kshitiz/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True